In [ ]:
import glob 
import numpy as np
from keras import optimizers
from keras.models import Model
import matplotlib.pyplot as plt 
from keras.models import Sequential
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, concatenate, Conv2D, BatchNormalization, Activation, SeparableConv2D, MaxPooling2D, add
from keras.layers import Input, Dense

X_CLAHE = np.load('/content/drive/MyDrive/Database/covid19_data/covid19_clahe/clahe_288_X_0_255.npy')
X255 = np.load('/content/drive/MyDrive/Database/covid19_data/covid19_original/original_X_0_255.npy')

y_ = np.load('/content/drive/MyDrive/Database/covid19_data/covid19_original/original_y_0_255.npy')
# print(y_)
# print(X_CLAHE[:,:,:,:1].shape)
X_CLAHE = X_CLAHE[:,:,:,:1]/255
X255 = X255[:,:,:,:1]/255

# print(X_CLAHE.shape, X255.shape)
X_CLAHE = X_CLAHE.astype("float16")
X255 = X255.astype("float16")



In [ ]:
print(y_)

[0 0 0 ... 2 2 2]


In [ ]:
X1 = X255
X1 = np.concatenate((X_CLAHE, X255), axis=3)
print(X1.shape)


from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y_, test_size=0.2, random_state=10, stratify = y_)
X_test1, X_val1, y_test1, y_val1 = train_test_split(X_test1, y_test1, test_size=0.5, random_state=5, stratify = y_test1)
print(X_test1.shape)

(3875, 224, 224, 2)
(387, 224, 224, 2)


In [ ]:
for i in range(len(y_train1)):
  if y_train1[i] == 2:
    y_train1[i] = 1

for i in range(len(y_test1)):
  if y_test1[i] == 2:
    y_test1[i] = 1
  
for i in range(len(y_val1)):
  if y_val1[i] == 2:
    y_val1[i] = 1

In [ ]:
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet201
# from keras.applications.inception_resnet_v2 import InceptionResNetV2

model = DenseNet201(weights=None, include_top=False, input_shape=[224, 224, X1.shape[3]])
# filepath="/content/drive/MyDrive/Database/covid19_data/2classification/2classification_Xception_original"
# filepath="/content/drive/MyDrive/Database/covid19_data/2classification/2classification_Xception_clahe"
filepath="//content/drive/MyDrive/Database/covid19_data/2classification/DenseNet201_0815"

model_x = model.output
x = GlobalAveragePooling2D(name='avg_pool')(model_x)
predictions_ = Dense(1, activation='sigmoid', name='predictions')(x)
model_ = Model(inputs=model.input, outputs=predictions_)
model_.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/applications/imagenet_utils.py:333: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 2) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 576         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

checkpoint = ModelCheckpoint(filepath+'_best.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto', save_weights_only = False)
callbacks_list = [checkpoint]
# LR_function=ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.0001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0000001, verbose=1)
stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
opt = optimizers.Adam(lr=0.1)
model_.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])
history = model_.fit(X_train1, y_train1, batch_size = 16, epochs = 1000,validation_data =(X_val1, y_val1), shuffle=True, callbacks=[reduce_lr, checkpoint, stop])

score = model_.evaluate(X_test1, y_test1)
print(score[1])

model_.save(filepath+'_last.h5')

Epoch 1/1000
194/194 [==============================] - 71s 299ms/step - loss: 1.4035 - accuracy: 0.7836 - val_loss: 0.1784 - val_accuracy: 0.9381

Epoch 00001: val_accuracy improved from -inf to 0.93814, saving model to //content/drive/MyDrive/Database/covid19_data/2classification/run_new/2classification_Xception_2ch_more_epochs_best.h5
Epoch 2/1000
194/194 [==============================] - 56s 289ms/step - loss: 0.1014 - accuracy: 0.9639 - val_loss: 1.1825 - val_accuracy: 0.8479

Epoch 00002: val_accuracy did not improve from 0.93814
Epoch 3/1000
194/194 [==============================] - 57s 293ms/step - loss: 0.0767 - accuracy: 0.9738 - val_loss: 0.4381 - val_accuracy: 0.9330

Epoch 00003: val_accuracy did not improve from 0.93814
Epoch 4/1000
194/194 [==============================] - 58s 297ms/step - loss: 0.0675 - accuracy: 0.9789 - val_loss: 0.1015 - val_accuracy: 0.9820

Epoch 00004: val_accuracy improved from 0.93814 to 0.98196, saving model to //content/drive/MyDrive/Databa

KeyboardInterrupt: ignored

In [ ]:
# plt.title(score[0])
plt.title('loss')
plt.plot(history.history["loss"], label = "Train Loss")
plt.plot(history.history["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()
# plt.savefig("/content/drive/MyDrive/Database/covid19_data/2classification/run_new/2_classification.tif" ,dpi = 300)

# plt.title(score[1])
plt.title('accuracy')
plt.plot(history.history["accuracy"], label = "Train accuracy")
plt.plot(history.history["val_accuracy"], label = "Validation accuracy")
plt.legend()
plt.show()
# plt.savefig("/content/drive/MyDrive/Database/covid19_data/2classification/run_new/2_classification.tif" ,dpi = 300)

In [ ]:

csfont = {'fontname':'Times New Roman'}


plt.title('Training and Validation Accuracy', **csfont)
# plt.rc('legend', fontsize=9)
plt.plot(history.history["accuracy"])#, label = "Training Accuracy")
plt.plot(history.history["val_accuracy"])#, label = "Validation Accuracy")
plt.ylabel('Accuracy', **csfont)
plt.xlabel('Epoch', **csfont)
plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
plt.show()
# plt.savefig("/content/drive/MyDrive/Database/covid19_data/2classification/run_new/二分類準確度_有title.tif" ,dpi = 300)


# plt.figure(dpi=300)
plt.title('Training and Validation Loss', **csfont)
# plt.rc('legend', fontsize=9)
plt.plot(history.history['loss'])#, label = "Training Loss")
plt.plot(history.history['val_loss'])#, label = "Validation Loss")
plt.ylabel('Loss', **csfont)
plt.xlabel('Epoch', **csfont)
# plt.legend(['Training Loss', 'Validation Accuracy'], loc='upper right')
plt.show()
# plt.savefig("/content/drive/MyDrive/Database/covid19_data/2classification/run_new/二分類Loss_有title.tif" ,dpi = 300)


In [ ]:
print(history.history.keys())
history.history['loss']
history.history['accuracy']
history.history['val_loss']
history.history['val_accuracy']
history.history['lr']
import pandas as pd

mid_term_marks = {"loss":history.history['loss'],
          "accuracy":history.history['accuracy'],
          "val_loss":history.history['val_loss'],
          "val_accuracy":history.history['val_accuracy'],
          "lr":history.history['lr']}

mid_term_marks_df = pd.DataFrame(mid_term_marks)
mid_term_marks_df.to_csv("/content/drive/MyDrive/Database/covid19_data/2classification/run_new/二分類_more_epochs.csv")

In [ ]:
ans = []
predict = model_.predict(X_test1)
# predict = np.argmax(predict,axis=1)

print(predict.shape)
for i in range(len(predict)):
    if predict[i] > 0.5:
        predict[i] = 1
        ans.append(1)
    else:
        predict[i] = 0
        ans.append(0)

In [ ]:
ans = np.array(ans)
import pandas as pd
print(filepath+' last.h5')
print(pd.crosstab(y_test1, ans, rownames=['Actual'],colnames=['prediction']))
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test1, ans)
# print(matrix)
# TP FN
# FP TN
TP = matrix[0][0]
FN = matrix[0][1]
FP = matrix[1][0]
TN = matrix[1][1]
Precision = TP/(TP+FP)
recall_Sensitivity = TP/(TP+FN)
F1 = (2*Precision*recall_Sensitivity)/(recall_Sensitivity+Precision)
print("Accuracy:",(TP+TN)/(TP+TN+FP+FN))
print("Precision:",Precision)
print("Sensitivity:",TP/(TP+FN))
print("F1", F1)
print("Specificity", TN/(TN+FP))

from tensorflow import keras

model_load = keras.models.load_model(filepath+'_best.h5')
print(filepath+' best.h5')
ans_p = []
predict = model_load.predict(X_test1)
for i in range(len(predict)):
    if predict[i] > 0.5:
        predict[i] = 1
        ans_p.append(1)
    else:
        predict[i] = 0
        ans_p.append(0)
        
ans_p = np.array(ans_p)
import pandas as pd
print(pd.crosstab(y_test1, ans_p, rownames=['Actual'],colnames=['prediction']))

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test1, ans_p)
# print(matrix)
# TP FN
# FP TN
TP = matrix[0][0]
FN = matrix[0][1]
FP = matrix[1][0]
TN = matrix[1][1]
Precision = TP/(TP+FP)
recall_Sensitivity = TP/(TP+FN)
F1 = (2*Precision*recall_Sensitivity)/(recall_Sensitivity+Precision)
print("Accuracy:",(TP+TN)/(TP+TN+FP+FN))
print("Precision:",Precision)
print("Sensitivity:",TP/(TP+FN))
print("F1", F1)
print("Specificity", TN/(TN+FP))